In [117]:
import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np
from holoviews import opts
import pathlib
import holoviews as hv
from scipy.linalg import eigvals, eig

pn.extension(design='material')
pn.extension('katex', 'mathjax')

In [118]:
range_min = -5.0
range_max = 5.0
step_size = 0.5
xs, ys = np.arange(range_min, range_max, step_size), np.arange(range_min, range_max, step_size)
X, Y = np.meshgrid(xs, ys)
def dx(a, b):
    return a*X + b*Y
def dy(c, d):
    return c*X + d*Y

def phase_portrait(a, b, c, d):
    U = dx(a, b)
    V = dy(c, d)
    return U, V


In [119]:
def create_phase_plot(a, b, c, d):
    U,V = phase_portrait(a, b, c, d)
    mag = np.sqrt(U**2 + V**2)
    angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)
    opts.VectorField(height=800, width=800), opts.Curve(height=800, width=800), 
    opts.defaults(opts.Scatter(color="red",size=10))

    vectorfield = hv.VectorField((xs, ys, angle, mag)).opts(shared_axes=False)
    # find any eigenvectors
    matrix = np.array([[a, b], [c,d]])
    _, eigs = eig(matrix)
    ds = hv.Dataset(np.linspace(range_min, range_max), 'x')
    
    # if the eigenvalues are not complex, find straight-line solutions
    if True in np.iscomplex(eigs):
        return vectorfield
    else:
        slope_1 = eigs[1][0] / eigs[0][0]
        slope_2 = eigs[1][1] / eigs[0][1]

        expr1 = (hv.dim('x')*slope_1)
        expr2 = (hv.dim('x')*slope_2)

        sl_1 = ds.transform(y=expr1)
        sl_2 = ds.transform(y=expr2)

        sl_curve1 = hv.Curve(sl_1).opts(xlim=(range_min, range_max), ylim=(range_min, range_max), shared_axes=False, color="blue")
        sl_curve2 = hv.Curve(sl_2).opts(xlim=(range_min, range_max), ylim=(range_min, range_max), shared_axes=False, color="blue")
        out = vectorfield*sl_curve1*sl_curve2
        out.opts(shared_axes = False)
        #vectorfield.relabel(label)
        return out

In [120]:
a_widget = pn.widgets.FloatSlider(name="a", value = 1, start = range_min, end=range_max)
b_widget = pn.widgets.FloatSlider(name="b", value = 1, start = range_min, end=range_max)
c_widget = pn.widgets.FloatSlider(name="c", value = 1, start = range_min, end=range_max)
d_widget = pn.widgets.FloatSlider(name="d", value = 1, start = range_min, end=range_max)


In [121]:
bound_plot = pn.bind(create_phase_plot, a=a_widget, b=b_widget, c=c_widget, d=d_widget)

In [122]:
ds = hv.Dataset(np.linspace(-10, 10), 't')
expr = (hv.dim('t')**2)/4
transformed = ds.transform(y=expr)

def trace(a, b, c, d):
    return a + d

def determinant(a, b, c, d):
    return a*d-b*c

def td_point(a, b, c, d):   
    first = hv.Curve(transformed).opts(shared_axes=False)
    second =  hv.Scatter([(trace(a, b, c, d), determinant(a, b, c, d))]).opts(shared_axes=False)
    return first*second



def eigenvalues(a,b,c,d):
    matrix = np.array([[a, b], [c,d]])
    eigs = eigvals(matrix)
    if (eigs[0].imag != 0j) or (eigs[1].imag !=0j):
        return pn.Column(pn.pane.LaTeX(r"$\lambda_1={:.3f}$".format(eigs[1])), pn.pane.LaTeX(r"$\lambda_2={:.3f}$".format(eigs[0])))
    else:
        return pn.Column(pn.pane.LaTeX(r"$\lambda_1={:.3f}$".format(eigs[1].real)), pn.pane.LaTeX(r"$\lambda_2={:.3f}$".format(eigs[0].real)))
    


In [123]:
td_plot = pn.bind(td_point, a=a_widget, b=b_widget, c=c_widget, d=d_widget)

In [124]:
eigen_indicator = pn.bind(eigenvalues, a=a_widget, b=b_widget, c=c_widget, d=d_widget)

In [125]:
test_app.servable()

Column(design=<class 'panel.theme.materi...)
    [0] Row(design=<class 'panel.theme.materi...)
        [0] Column(design=<class 'panel.theme.materi...)
            [0] FloatSlider(design=<class 'panel.theme.materi..., end=5.0, name='a', start=-5.0, value=1)
            [1] FloatSlider(design=<class 'panel.theme.materi..., end=5.0, name='b', start=-5.0, value=1)
            [2] FloatSlider(design=<class 'panel.theme.materi..., end=5.0, name='c', start=-5.0, value=1)
            [3] FloatSlider(design=<class 'panel.theme.materi..., end=5.0, name='d', start=-5.0, value=1)
        [1] ParamFunction(function, _pane=Column, defer_load=False, design=<class 'panel.theme.materi...)
    [1] Row(design=<class 'panel.theme.materi...)
        [0] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi...)
        [1] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi...)